In [ ]:
%matplotlib inline

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../nn-classifiers/'))

In [ ]:
import labeling_network as lbln
from matplotlib import pyplot as plt
import theano
import theano.tensor as T
import numpy as np
from labeling_network import FullyConnectedLayer, ConvPoolLayer

In [ ]:
dataPath = '../../../factory-robot-data/imgs_2015-09-28/'

In [ ]:
train_data, valid_data, test_data = lbln.load_data_shared(
                     dataPath,
                     'capture',
                     dataPath + 'labels.dat',
                     n_train=100,
                     n_validation=1000,
                     n_test=10,
                     label_mask=1)

# Modify data tuples for autoencoder learning. Target output = input.
train_data = train_data[0], train_data[0]
valid_data = valid_data[0], valid_data[0]
test_data = test_data[0], test_data[0]

In [ ]:
mini_batch_size = 10

### Train the Network. 

In [ ]:
network = lbln.Network([

        FullyConnectedLayer(n_in=64*64*3, n_out=32*32, p_dropout=0.0),
        FullyConnectedLayer(n_in=32*32, n_out=64, p_dropout=0.0),
        FullyConnectedLayer(n_in=64, n_out=64*64*3),
        
    ], mini_batch_size)

In [ ]:
network.SGD(train_data, 
            40, 
            mini_batch_size, 
            10.0, 
            valid_data, 
            test_data,
            "one_layer_simple_ac",
#             rmsprop=(0.001, 0.9, 1e-6, 20.0)
           )

#### Save network weights.

In [ ]:
network.save_as_file('ac_')

In [ ]:
train_data_xs = None #train_data[0].get_value()
train_data_ts = None #train_data[1].get_value()

test_data_xs = test_data[0].get_value()
test_data_ts = test_data[1].get_value()

valid_data_xs = valid_data[0].get_value()
valid_data_ts = valid_data[1].get_value()

In [ ]:
index = 2
inpt = test_data_xs[index]
output = network.get_single_output(inpt)
print 'error:', np.mean((output - inpt) ** 2)

# randomout = np.zeros(inpt.shape) + 0.5

# print np.mean((randomout - inpt) ** 2)


# print 'mean:', np.mean(train_data_xs)
# print 'sq diff:', np.mean((inpt - np.mean(train_data_xs)) ** 2)

plt.imshow(inpt.reshape((64, 64, 3)), interpolation='nearest')
plt.show()
plt.imshow(output.reshape((64, 64, 3)), interpolation='nearest')
plt.show()

In [ ]:
loaded_net = lbln.Network.load_from_file('ac', mini_batch_size)

In [ ]:
loaded_net.single_output = loaded_net.layers[-2].single_output

In [ ]:
for index in xrange(10):
    inpt = test_data_xs[index]
    output = loaded_net.get_single_output(inpt)
    print 'error:', np.mean((output - inpt) ** 2)

    plt.imshow(inpt.reshape((64, 64, 3)), interpolation='nearest')
    plt.show()
    plt.imshow(output.reshape((64, 64, 3)), interpolation='nearest')
    plt.show()

    # print output.shape